In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.layers import Input, Conv1D, Flatten, Dense, Dropout, BatchNormalization, MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')

def create_features(df):
    """Create features from CAN bus data"""
    # Convert hex to numeric
    df['CAN_ID_numeric'] = df['CAN_ID_hex'].apply(lambda x: int(str(x), 16) if pd.notna(x) else 0)
    
    # Process data payload
    df['Data_clean'] = df['Data_hex'].str.replace(' ', '').fillna('00000000')
    
    # Extract bytes
    for i in range(8):
        df[f'Byte_{i}'] = df['Data_clean'].apply(
            lambda x: int(x[i*2:(i+1)*2], 16) if len(x) > i*2+1 else 0
        )
    
    return df

def load_data():
    """Load and prepare data"""
    file_paths = [
        "/Users/bodapati/Downloads/Car_Hacking_Challenge_Dataset_rev20Mar2021/0_Preliminary/0_Training/Pre_train_D_0.csv",
   "/Users/bodapati/Downloads/Car_Hacking_Challenge_Dataset_rev20Mar2021/0_Preliminary/0_Training/Pre_train_D_1.csv",
   "/Users/bodapati/Downloads/Car_Hacking_Challenge_Dataset_rev20Mar2021/0_Preliminary/0_Training/Pre_train_D_2.csv",
   "/Users/bodapati/Downloads/Car_Hacking_Challenge_Dataset_rev20Mar2021/0_Preliminary/0_Training/Pre_train_S_0.csv",
   "/Users/bodapati/Downloads/Car_Hacking_Challenge_Dataset_rev20Mar2021/0_Preliminary/0_Training/Pre_train_S_1.csv",
    "/Users/bodapati/Downloads/Car_Hacking_Challenge_Dataset_rev20Mar2021/0_Preliminary/0_Training/Pre_train_S_2.csv"
    ]
    all_data = []
    for path in file_paths:
        try:
            df = pd.read_csv(path)
            all_data.append(df)
            print(f"✅ Loaded {len(df)} samples from {path.split('/')[-1]}")
        except FileNotFoundError:
            print(f"⚠️ File not found: {path}")
    
    if not all_data:
        return None, None
    
    # Combine data
    df = pd.concat(all_data, ignore_index=True)
    
    # Standardize column names
    df.rename(columns={
        'Arbitration_ID': 'CAN_ID_hex',
        'Data': 'Data_hex'
    }, inplace=True)
    
    # Create features
    df = create_features(df)
    
    # Prepare target variable (binary: Attack vs Normal)
    print(f"Available classes in 'Class' column: {df['Class'].unique()}")
    if 'SubClass' in df.columns:
        print(f"Available subclasses: {df['SubClass'].unique()}")
    
    # Create binary label
    if 'R' in df['Class'].values:
        df['Binary_Label'] = df['Class'].apply(lambda x: 1 if x == 'R' else 0)
    elif 'Attack' in df['Class'].values:
        df['Binary_Label'] = df['Class'].apply(lambda x: 1 if x == 'Attack' else 0)
    elif 'SubClass' in df.columns:
        df['Binary_Label'] = df['SubClass'].apply(lambda x: 0 if pd.isna(x) or x == 'Normal' else 1)
    else:
        print("⚠️ No clear attack labels found. Creating synthetic labels for testing.")
        df['Binary_Label'] = np.random.choice([0, 1], size=len(df), p=[0.7, 0.3])
    
    # Check class distribution
    class_dist = df['Binary_Label'].value_counts()
    print(f"Class distribution: {dict(class_dist)}")
    
    # Ensure we have both classes
    if len(class_dist) < 2:
        print("⚠️ Only one class found. Adding minority class samples.")
        minority_class = 1 if class_dist.index[0] == 0 else 0
        n_minority = max(10, len(df) // 20)
        minority_indices = np.random.choice(len(df), size=n_minority, replace=False)
        df.loc[minority_indices, 'Binary_Label'] = minority_class
        print(f"Updated class distribution: {dict(df['Binary_Label'].value_counts())}")
    
    # Extract byte features for DCNN
    byte_features = [f'Byte_{i}' for i in range(8)]
    X = df[byte_features].fillna(0).values.astype(np.float32)
    y = df['Binary_Label'].values.astype(np.int32)
    
    print(f"\nDataset: {len(X)} samples, {X.shape[1]} features")
    print(f"Normal (0): {sum(y == 0)} ({sum(y == 0)/len(y)*100:.2f}%)")
    print(f"Attack (1): {sum(y == 1)} ({sum(y == 1)/len(y)*100:.2f}%)")
    
    return X, y

def build_dcnn(input_shape):
    """Build Deep Convolutional Neural Network for CAN intrusion detection"""
    inp = Input(shape=input_shape, name='input_layer')
    
    # Batch normalization for input
    x = BatchNormalization(name='bn_input')(inp)
    
    # First convolutional block
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same', name='conv1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = MaxPooling1D(pool_size=2, name='pool1')(x)
    x = Dropout(0.25, name='dropout1')(x)
    
    # Second convolutional block
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same', name='conv2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = Dropout(0.25, name='dropout2')(x)
    
    # Flatten and dense layers
    x = Flatten(name='flatten')(x)
    x = Dense(128, activation='relu', name='dense1')(x)
    x = Dropout(0.4, name='dropout3')(x)
    x = Dense(64, activation='relu', name='dense2')(x)
    x = Dropout(0.3, name='dropout4')(x)
    
    # Output layer
    out = Dense(2, activation='softmax', name='output')(x)
    
    model = Model(inputs=inp, outputs=out, name='DCNN_CAN_IDS')
    
    # Compile with optimizer
    model.compile(
        loss='categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics=['accuracy']
    )
    
    return model

def train_dcnn():
    """Train DCNN model"""
    print("🚗 DCNN Car Hacking Detection")
    print("=" * 40)
    
    # Load data
    X, y = load_data()
    if X is None:
        print("❌ Failed to load data")
        return None, 0
    
    # Reshape for Conv1D: (samples, timesteps, features)
    X = X.reshape(-1, 8, 1)
    
    # Convert labels to categorical
    y_cat = to_categorical(y, num_classes=2)
    
    # Split data with stratification
    try:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_cat, test_size=0.2, random_state=42, stratify=y
        )
    except ValueError:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_cat, test_size=0.2, random_state=42
        )
    
    print(f"\nTrain: {len(X_train)}, Test: {len(X_test)}")
    print(f"Train class distribution: Normal={sum(np.argmax(y_train, axis=1) == 0)}, Attack={sum(np.argmax(y_train, axis=1) == 1)}")
    print(f"Test class distribution: Normal={sum(np.argmax(y_test, axis=1) == 0)}, Attack={sum(np.argmax(y_test, axis=1) == 1)}")
    
    # Final check for class balance
    if len(np.unique(np.argmax(y_train, axis=1))) < 2:
        print("❌ Training set has only one class. Cannot train classifier.")
        return None, 0
    
    # Build model
    print("\n🏗️  Building DCNN model...")
    model = build_dcnn((8, 1))
    
    print("\nModel Architecture:")
    model.summary()
    
    # Callbacks
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=0.00001,
        verbose=1
    )
    
    # Train model
    print("\n💡 Training DCNN...")
    history = model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=128,
        validation_data=(X_test, y_test),
        callbacks=[early_stop, reduce_lr],
        verbose=1
    )
    
    # Predictions
    y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
    y_test_label = np.argmax(y_test, axis=1)
    
    # Results
    accuracy = accuracy_score(y_test_label, y_pred)
    print(f"\n📈 Results:")
    print(f"Accuracy: {accuracy:.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(y_test_label, y_pred)
    print(f"\nConfusion Matrix:")
    print(f"              Predicted")
    print(f"            Normal  Attack")
    print(f"Normal  [{cm[0][0]:7,d} {cm[0][1]:7,d}]")
    print(f"Attack  [{cm[1][0]:7,d} {cm[1][1]:7,d}]")
    
    print(f"\nDetailed Report:")
    print(classification_report(
        y_test_label, y_pred,
        target_names=['Normal', 'Attack'],
        digits=4
    ))
    
    # Save model
    model_path = "dcnn_car_hacking_model.h5"
    model.save(model_path)
    print(f"\n💾 Model saved to: {model_path}")
    
    return model, accuracy

if __name__ == "__main__":
    model, accuracy = train_dcnn()
    
    if model is not None:
        print(f"\n🎯 Final Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    else:
        print("\n❌ Training failed.")

🚗 DCNN Car Hacking Detection
✅ Loaded 179346 samples from C:\Users\nagas\Downloads\Car_Hacking_Challenge_Dataset_rev20Mar2021\0_Preliminary\0_Training\Pre_train_D_0.csv
✅ Loaded 806390 samples from C:\Users\nagas\Downloads\Car_Hacking_Challenge_Dataset_rev20Mar2021\0_Preliminary\0_Training\Pre_train_D_1.csv
✅ Loaded 889395 samples from C:\Users\nagas\Downloads\Car_Hacking_Challenge_Dataset_rev20Mar2021\0_Preliminary\0_Training\Pre_train_D_2.csv
✅ Loaded 180686 samples from C:\Users\nagas\Downloads\Car_Hacking_Challenge_Dataset_rev20Mar2021\0_Preliminary\0_Training\Pre_train_S_0.csv
✅ Loaded 799292 samples from C:\Users\nagas\Downloads\Car_Hacking_Challenge_Dataset_rev20Mar2021\0_Preliminary\0_Training\Pre_train_S_1.csv
✅ Loaded 817042 samples from C:\Users\nagas\Downloads\Car_Hacking_Challenge_Dataset_rev20Mar2021\0_Preliminary\0_Training\Pre_train_S_2.csv
Available classes in 'Class' column: ['Normal' 'Attack']
Available subclasses: [nan 'Normal' 'Flooding' 'Spoofing' 'Replay' 'Fuzzin

Model: "DCNN_CAN_IDS"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 8, 1)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn_input (BatchNormalization)   │ (None, 8, 1)           │             4 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv1D)                  │ (None, 8, 64)          │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 8, 64)          │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1 (MaxPooling1D)            │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1 (Dropout)              │ (None, 4, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv1D)                  │ (None, 4, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 4, 128)         │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout2 (Dropout)              │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout3 (Dropout)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout4 (Dropout)              │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,782 (389.77 KB)

 Trainable params: 99,396 (388.27 KB)

 Non-trainable params: 386 (1.51 KB)


💡 Training DCNN...
Epoch 1/30
22951/22951 ━━━━━━━━━━━━━━━━━━━━ 181s 8ms/step - accuracy: 0.9504 - loss: 0.1311 - val_accuracy: 0.9606 - val_loss: 0.1083 - learning_rate: 0.0010
Epoch 2/30
22951/22951 ━━━━━━━━━━━━━━━━━━━━ 180s 8ms/step - accuracy: 0.9573 - loss: 0.1153 - val_accuracy: 0.9610 - val_loss: 0.1063 - learning_rate: 0.0010
Epoch 3/30
22951/22951 ━━━━━━━━━━━━━━━━━━━━ 180s 8ms/step - accuracy: 0.9590 - loss: 0.1119 - val_accuracy: 0.9636 - val_loss: 0.1029 - learning_rate: 0.0010
Epoch 4/30
22951/22951 ━━━━━━━━━━━━━━━━━━━━ 179s 8ms/step - accuracy: 0.9599 - loss: 0.1101 - val_accuracy: 0.9643 - val_loss: 0.1018 - learning_rate: 0.0010
Epoch 5/30
22951/22951 ━━━━━━━━━━━━━━━━━━━━ 182s 8ms/step - accuracy: 0.9605 - loss: 0.1091 - val_accuracy: 0.9643 - val_loss: 0.1016 - learning_rate: 0.0010
Epoch 6/30
22951/22951 ━━━━━━━━━━━━━━━━━━━━ 185s 8ms/step - accuracy: 0.9609 - loss: 0.1084 - val_accuracy: 0.9632 - val_loss: 0.1027 - learning_rate: 0.0010
Epoch 7/30
22951/22951 ━━━━━━━━━


💾 Model saved to: dcnn_car_hacking_model.h5

🎯 Final Accuracy: 0.9648 (96.48%)
